<a href="https://colab.research.google.com/github/rohilahuja3/Ai_Final_labs/blob/main/8_puzzle_using_DFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class eight_puzzle_probolem():

    def __init__(self):
        self.goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, -1]]
        self.size = 3

    def print_the_matrix(self, state):
        for list in state:
            for ele in list:
                print(ele, end=" ")
            print()
        print("====")

    def check_if_location_is_inside_the_matrix_or_not(self, current_location):
        x, y = current_location
        if(x<0 or y<0 or x>=self.size or y>=self.size):
            return False
        return True

    def find_location_of_space(self, current_state):
        for i in range(len(current_state)):
            for j in range(len(current_state[i])):
                if(current_state[i][j] == -1):
                    return i,j   #why don't we write about the case that -1 does not exist in the matrix

    def create_a_new_state_using_current_and_new_location(self, current_state, current_location, new_location):
        x1, y1 = current_location
        x2, y2 = new_location

        if(self.check_if_location_is_inside_the_matrix_or_not(new_location)):
            current_state[x1][y1], current_state[x2][y2] = current_state[x2][y2], current_state[x1][y1]

        return current_state

    def DFS(self, state):
        current_location=self.find_location_of_space(state)
        #print(current_location)

        visited_DFS={}
        stack_DFS=[current_location]
        count=0

        while (stack_DFS):
            new_location=stack_DFS.pop()
            state=self.create_a_new_state_using_current_and_new_location(state, current_location, new_location)

            state_str=str(state)
            if(state_str == str(self.goal_state)):
                print("No. of states iterated: ")
                print(count)
                print("Goal state reached: ")
                self.print_the_matrix(state)
                #print(visited_DFS)
                return

            self.print_the_matrix(state)
            count+=1

            if(not visited_DFS.get(state_str, False)):
                x,y=new_location

                #check if we can go up
                if(self.check_if_location_is_inside_the_matrix_or_not((x-1,y))):
                    stack_DFS.append((x-1,y))

                #check if we can go down
                if(self.check_if_location_is_inside_the_matrix_or_not((x+1,y))):
                    stack_DFS.append((x+1,y))

                #check if we can go right
                if(self.check_if_location_is_inside_the_matrix_or_not((x,y+1))):
                    stack_DFS.append((x,y+1))

                #check if we can go left
                if(self.check_if_location_is_inside_the_matrix_or_not((x,y-1))):
                    stack_DFS.append((x,y-1))

            else:
                state=self.create_a_new_state_using_current_and_new_location(state, new_location, current_location)

            current_location=self.find_location_of_space(state)
            visited_DFS[state_str]=True
        


g=eight_puzzle_probolem()
initial_state = [[-1, 5, 2], [1, 8, 3], [4, 7, 6]]
# initial_state=[[1,2,3],[8,-1,4],[7,6,5]]
g.DFS(initial_state)

In [72]:
import numpy as np
import time

class Node():
    def __init__(self,state,parent,action,depth):
        self.state = state 
        self.parent = parent # parent node
        self.action = action # move up, left, down, right
        self.depth = depth # depth of the node in the tree
        
        # children node
        self.move_up = None
        self.move_left = None
        self.move_down = None
        self.move_right = None
    
    # see if moving down is valid
    def try_move_down(self):
        # index of the empty tile
        zero_index=[i[0] for i in np.where(self.state==0)] 
        if zero_index[0] == 0:
            return False
        else:
            up_value = self.state[zero_index[0]-1,zero_index[1]] # value of the upper tile
            new_state = self.state.copy()
            new_state[zero_index[0],zero_index[1]] = up_value
            new_state[zero_index[0]-1,zero_index[1]] = 0
            return new_state,up_value
        
    # see if moving right is valid
    def try_move_right(self):
        zero_index=[i[0] for i in np.where(self.state==0)] 
        if zero_index[1] == 0:
            return False
        else:
            left_value = self.state[zero_index[0],zero_index[1]-1] # value of the left tile
            new_state = self.state.copy()
            new_state[zero_index[0],zero_index[1]] = left_value
            new_state[zero_index[0],zero_index[1]-1] = 0
            return new_state,left_value
        
    # see if moving up is valid
    def try_move_up(self):
        zero_index=[i[0] for i in np.where(self.state==0)] 
        if zero_index[0] == 2:
            return False
        else:
            lower_value = self.state[zero_index[0]+1,zero_index[1]] # value of the lower tile
            new_state = self.state.copy()
            new_state[zero_index[0],zero_index[1]] = lower_value
            new_state[zero_index[0]+1,zero_index[1]] = 0
            return new_state,lower_value
        
    # see if moving left is valid
    def try_move_left(self):
        zero_index=[i[0] for i in np.where(self.state==0)] 
        if zero_index[1] == 2:
            return False
        else:
            right_value = self.state[zero_index[0],zero_index[1]+1] # value of the right tile
            new_state = self.state.copy()
            new_state[zero_index[0],zero_index[1]] = right_value
            new_state[zero_index[0],zero_index[1]+1] = 0
            return new_state,right_value
        
    # once the goal node is found, trace back to the root node and print out the path
    def print_path(self):
        # create FILO stacks to place the trace
        state_trace = [self.state]
        action_trace = [self.action]
        depth_trace = [self.depth]
        
        # add node information as tracing back up the tree
        while self.parent:
            self = self.parent

            state_trace.append(self.state)
            action_trace.append(self.action)
            depth_trace.append(self.depth)

        # print out the path
        step_counter = 0
        while state_trace:
            print('step',step_counter)
            print(state_trace.pop())
            print('action=',action_trace.pop(),', depth=',str(depth_trace.pop()),'\n')
            
            step_counter += 1    
                
    def depth_first_search(self, goal_state):
        start = time.time()
        
        queue = [self] # queue of found but unvisited nodes, FILO
        queue_num_nodes_popped = 0 # number of nodes popped off the queue, measuring time performance
        queue_max_length = 1 # max number of nodes in the queue, measuring space performance
        
        depth_queue = [0] # queue of node depth
        path_cost_queue = [0] # queue for path cost
        visited = set([]) # record visited states
        
        while queue:
            # update maximum length of the queue
            if len(queue) > queue_max_length:
                queue_max_length = len(queue)
                
            current_node = queue.pop(0) # select and remove the first node in the queue
            queue_num_nodes_popped += 1
            
            current_depth = depth_queue.pop(0) # select and remove the depth for current node
            current_path_cost = path_cost_queue.pop(0) # select and remove the path cost for reaching current node
            visited.add(tuple(current_node.state.reshape(1,9)[0])) # add state, which is represented as a tuple
            
            # when the goal state is found, trace back to the root node and print out the path
            if np.array_equal(current_node.state,goal_state):
                current_node.print_path()
                print("Goal reached")
                return
            
            else:                
                # see if moving upper tile down is a valid move
                if current_node.try_move_down():
                    new_state,up_value = current_node.try_move_down()
                    # check if the resulting node is already visited
                    if tuple(new_state.reshape(1,9)[0]) not in visited:
                        # create a new child node
                        current_node.move_down = Node(state=new_state,parent=current_node,action='down',depth=current_depth+1)
                        queue.insert(0,current_node.move_down)
                        depth_queue.insert(0,current_depth+1)
                        path_cost_queue.insert(0,current_path_cost+up_value)
                    
                # see if moving left tile to the right is a valid move
                if current_node.try_move_right():
                    new_state,left_value = current_node.try_move_right()
                    # check if the resulting node is already visited
                    if tuple(new_state.reshape(1,9)[0]) not in visited:
                        # create a new child node
                        current_node.move_right = Node(state=new_state,parent=current_node,action='right',depth=current_depth+1)
                        queue.insert(0,current_node.move_right)
                        depth_queue.insert(0,current_depth+1)
                        path_cost_queue.insert(0,current_path_cost+left_value)
                 
                # see if moving lower tile up is a valid move
                if current_node.try_move_up():
                    new_state,lower_value = current_node.try_move_up()
                    # check if the resulting node is already visited
                    if tuple(new_state.reshape(1,9)[0]) not in visited:
                        # create a new child node
                        current_node.move_up = Node(state=new_state,parent=current_node,action='up',depth=current_depth+1)
                        queue.insert(0,current_node.move_up)
                        depth_queue.insert(0,current_depth+1)
                        path_cost_queue.insert(0,current_path_cost+lower_value)

                # see if moving right tile to the left is a valid move
                if current_node.try_move_left():
                    new_state,right_value = current_node.try_move_left()
                    # check if the resulting node is already visited
                    if tuple(new_state.reshape(1,9)[0]) not in visited:
                        # create a new child node
                        current_node.move_left = Node(state=new_state,parent=current_node,action='left',depth=current_depth+1)
                        queue.insert(0,current_node.move_left)
                        depth_queue.insert(0,current_depth+1)
                        path_cost_queue.insert(0,current_path_cost+right_value)

In [73]:
initial_state = np.array([[0, 5, 2], [1, 8, 3], [4, 7, 6]])
# initial_state = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
goal_state = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
print(initial_state,'\n')
print(goal_state)

root_node = Node(state=initial_state,parent=None,action=None,depth=0)
root_node.depth_first_search(goal_state)

Streaming output truncated to the last 5000 lines.

step 364
[[3 8 7]
 [4 0 5]
 [2 1 6]]
action= right , depth= 364 

step 365
[[3 8 7]
 [4 1 5]
 [2 0 6]]
action= up , depth= 365 

step 366
[[3 8 7]
 [4 1 5]
 [2 6 0]]
action= left , depth= 366 

step 367
[[3 8 7]
 [4 1 0]
 [2 6 5]]
action= down , depth= 367 

step 368
[[3 8 7]
 [4 0 1]
 [2 6 5]]
action= right , depth= 368 

step 369
[[3 8 7]
 [0 4 1]
 [2 6 5]]
action= right , depth= 369 

step 370
[[3 8 7]
 [2 4 1]
 [0 6 5]]
action= up , depth= 370 

step 371
[[3 8 7]
 [2 4 1]
 [6 0 5]]
action= left , depth= 371 

step 372
[[3 8 7]
 [2 4 1]
 [6 5 0]]
action= left , depth= 372 

step 373
[[3 8 7]
 [2 4 0]
 [6 5 1]]
action= down , depth= 373 

step 374
[[3 8 7]
 [2 0 4]
 [6 5 1]]
action= right , depth= 374 

step 375
[[3 8 7]
 [2 5 4]
 [6 0 1]]
action= up , depth= 375 

step 376
[[3 8 7]
 [2 5 4]
 [6 1 0]]
action= left , depth= 376 

step 377
[[3 8 7]
 [2 5 0]
 [6 1 4]]
action= down , depth= 377 

step 378
[[3 8 7]
 [2 0 5]
 [6 1 4]]
act